In [1]:
import torch
import Trainer
from network import NFM
import torch.utils.data as Data
from Utils.criteo_loader import getTestData, getTrainData

nfm_config = \
{
    'n_class':9,
    'linear_hidden1':2000,
    #'linear_hidden':100,#线性模型输出层（隐层个数）
    #'embed_input_dim':1001,#embed输入维度
    #'embed_dim': 100, # 用于控制稀疏特征经过Embedding层后的稠密特征大小，embed输出维度
    #'dnn_hidden_units': [100,11],#MLP隐层和输出层
    
    'dnn_hidden_units':[100,8],#MLP隐层
    'num_sparse_features_cols':10477,#the number of the gene columns
    'num_dense_features': 0,#dense features number
    'bi_dropout': 0.5,#Bi-Interaction 的dropout
    'num_epoch': 500,#训练epoch次数
    'batch_size': 16,#batch_size
    'lr': 1e-3,
    'l2_regularization': 1e-4,
    'device_id': 0,
    'use_cuda': False,
    'epoch':1000,
    
   
    'gene_name':'dataset/qiuguan/origin_800/gene_name.csv',
    'label_name':'dataset/qiuguan/origin_800/gene_label.csv'
    #'all':''
    #'title':'data/xiaoqiu_gene_5000/train/gene_5000_gene_name.csv',
    #'all':'data/xiaoqiu_gene_5000/train/gene_5000_label_name.csv'
}
#model definition
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0=nn.BatchNorm1d(3301)
        self.fc1 = nn.Linear(3301, 800)
        self.bn1= nn.BatchNorm1d(800)
        self.fc2 = nn.Linear(800, 100)
        self.bn2=nn.BatchNorm1d(100)
        self.fc3=nn.Linear(100,9)
        self.bn3=nn.BatchNorm1d(9)
        
        self.drop=nn.Dropout(0.5)
    def forward(self, x):
        x=self.bn0(x)
        x = F.relu(self.drop(self.bn1(self.fc1(x))))
        x = F.relu(self.drop(self.bn2(self.fc2(x))))
        return F.softmax(self.bn3(self.fc3(x)), dim=1)
model = MLP().cuda()
print(model)

import os
import time
import argparse
import numpy as np
import pandas as pd 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
#3from tensorboardX import SummaryWriter
import torch.nn.functional as F  # 激励函数的库
#import network
import config
import evaluate
import data_utils
#import Trainer

def one_hot_smoothing(labels, classes, label_smoothing=0.2):
    #n = len(labels)
    n=labels.shape[0]
    eoff = label_smoothing / classes
    output = np.ones((n, classes), dtype=np.float32) * eoff
    for row, label in enumerate(labels):
        output[row, label] = 1 - label_smoothing + eoff
        #print("row:",row,"label:",label)
    return output

def one_hot(labels, classes):
    n = len(labels)
    #eoff = label_smoothing / classes
    output = np.zeros((n, classes), dtype=np.float32)
    for row, label in enumerate(labels):
        output[row, label] = 1
        #print("row:",row,"label:",label)
    return output


import pandas as pd 

from sklearn.model_selection import train_test_split




def one_hot_smoothing(labels, classes, label_smoothing=0.2):
    #n = len(labels)
    n=labels.shape[0]
    eoff = label_smoothing / classes
    output = np.ones((n, classes), dtype=np.float32) * eoff
    for row, label in enumerate(labels):
        output[row, label] = 1 - label_smoothing + eoff
        #print("row:",row,"label:",label)
    return output

def one_hot(labels, classes):
    n = len(labels)
    #eoff = label_smoothing / classes
    output = np.zeros((n, classes), dtype=np.float32)
    for row, label in enumerate(labels):
        output[row, label] = 1
        #print("row:",row,"label:",label)
    return output

class KZDatasetTest(data.Dataset):
    """ Construct the FM pytorch dataset. """
    #def __init__(self, file,label_file, feature_map,n_class=16):
    def __init__(self, csv_path):
    
        self.data_info = self.get_data_info(csv_path)
        
        
            
        
        
        

    def __getitem__(self, index):
        # Dataset读取图片的函数
        data, label = self.data_info[index]
        #img = Image.open(img_pth).convert('RGB')
        
        return data, label

    def __len__(self):
        return len(self.data_info)
   
    
    
    def get_data_info(self,csv_path):
        #解析路径
        #转为一维list存储，每一位为【图片路径，图片类别】
        labels=[]
        data_info=[]
        df=pd.read_csv(csv_path,sep=',',header=None)
        
        df=df.iloc[1:,1:]
        #print("df:",df)
        #print(df.iloc[:,-1])
        #df=df.applymap(ast.literal_eval)
        rows,cols=df.shape
        #print(rows,cols)
        for i in df.iloc[:,-1]:
            #print(i)
            labels.append(int(i))
        #print('labels:',labels)
        labels=np.array(labels)
        #print('labels:',labels)
        #labels=np.array(labels)
        labels=one_hot_smoothing(labels,nfm_config['n_class'])
        #print(labels)
        for i in range(rows):
            data=df.iloc[i,:-1]
            data=data.astype(float)##############
            #print("i,data:",i,data)
            #data=pd.DataFrame(data,dtype=float)###############
            data=np.array(data)##
            
            label=labels[i]
            #print(data.shape)
            #print(label.shape)
            #label=label.tolist()
            data=torch.from_numpy(data)#
            label=torch.from_numpy(label)#
           
            
            data_info.append((data,label))
        return data_info
    
    
   
    
import torch
import torch.nn as nn
from torch.utils.data.dataset import *
from PIL import Image
from torch.nn import functional as F
import random
from sklearn.model_selection import train_test_split
import ast
import torchvision

MLP(
  (bn0): BatchNorm1d(3301, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=3301, out_features=800, bias=True)
  (bn1): BatchNorm1d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=800, out_features=100, bias=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=100, out_features=9, bias=True)
  (bn3): BatchNorm1d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)


In [2]:
#######找特征基因#############从3301中找200个基因
#########################################################本次测试的目的是看200个基因的分类效果
##########测试步骤：从3301个基因中提取350个
############用200个构建新的分类模型
#################特征基因
######################为小球，根据上边的测试的基因个数，350最大
import torch

import torch.nn as nn
import torch.optim as optim


from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from mnist_test import Net, train, test


# Network parameters
class Params(object):
    batch_size = 64
    test_batch_size = 20
    epochs = 5
    lr = 0.01
    momentum = 0.5
    no_cuda = True
    seed = 1
    log_interval = 10
    
    def __init__(self):
        pass

args = Params()
torch.manual_seed(args.seed)
#device = torch.device("cpu")
device=torch.device('cuda')
kwargs = {}



##############数据准备
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
#功能：加载保存到path中的各层参数到神经网络

path='dataset/qiuguan/model_new_K_fold_RandomTree/MLP_non_encode/MLP610.pkl'

#nfm=NFM(nfm_config)
mlp=MLP()
#print(nfm)
#net = nn.DataParallel(net)
#net = net.to(device)
mlp.load_state_dict(torch.load(path),strict=False)
mlp.cuda()

print(mlp)




mlp_params = list(mlp.named_parameters())
#print(nfm_params)
net=mlp
model=net###########
'''
testset = KZDatasetTest(csv_path='../NFM-pyorch-master/dataset/qiuguan/orign/')
   
test_loader = DataLoader(
         dataset=testset,
         #transform=torchvision.transforms.ToTensor(),
         
         batch_size=nfm_config['batch_size']
        
     )
'''

testset_xiaoqiu  = KZDatasetTest(csv_path='dataset/qiuguan/origin_800/xiaoqiu/test_info.csv')#样本收集特征数据集，和测试数据集不同，这里边可能还包含训练集
   
test_loader_xiaoqiu = DataLoader(
         dataset=testset_xiaoqiu,
         #transform=torchvision.transforms.ToTensor(),
         
         batch_size=nfm_config['batch_size'],
         shuffle=True
        
     )

testset_xiaoguan  = KZDatasetTest(csv_path='dataset/qiuguan/origin_800/xiaoguan/test_info.csv')
   
test_loader_xiaoguan = DataLoader(
         dataset=testset_xiaoguan,
         #transform=torchvision.transforms.ToTensor(),
         
         batch_size=nfm_config['batch_size'],
         shuffle=True
        
     )

################小球

#LRP
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline

from innvestigator import InnvestigateModel
from utils import Flatten


inn_model = InnvestigateModel(model, lrp_exponent=2,
                              method="e-rule",
                              beta=.5)

genes_features=np.array([i for i  in range(9)])
genes_features=genes_features.reshape(9,1).tolist()
model.double()
for data, target in test_loader_xiaoqiu:############小球

    data, target = data.to(device), target.to(device)
    #targets=torch.max(targets,1)[1]###################
    #print('data:',data.shape)
    batch_size = int(data.size()[0])
    #print('batch_size:',batch_size)#=20
    evidence_for_class = []
    #print("target:",target.shape)
    #print('target:',target[3])
    # Overlay with noise 
    # data[0] += 0.25 * data[0].max() * torch.Tensor(np.random.randn(28*28).reshape(1, 28, 28))
    #model_prediction, true_relevance = inn_model.innvestigate(in_tensor=data)

    for i in range(9):#10类
    # Unfortunately, we had some issue with freeing pytorch memory, therefore 
    # we need to reevaluate the model separately for every class.
        model_prediction, input_relevance_values = inn_model.innvestigate2(in_tensor=data, rel_for_class=i,target=target)
        evidence_for_class.append(input_relevance_values)
    #print('input_relevance_values:',input_relevance_values.shape)
    #print('evidence_for_class:',len(evidence_for_class))
    evidence_for_class = np.array([elt.numpy() for elt in evidence_for_class])
    #print('evidence_for_class:',evidence_for_class.shape)#[10,20,784]
    for idx, example in enumerate(data):#batch 中的每一个样本
        #print('example:',example.shape)
        prediction = np.argmax(model_prediction.cpu().detach(), axis=1)#
        #print('prediction[idx]:',prediction[idx])
        #print(evidence_for_class[prediction[idx]][idx])
        #fig, axes = plt.subplots(3, 5)
        '''
        fig.suptitle("Prediction of model: " + str(prediction[idx]) + "({0:.2f})".format(
            100*float(model_prediction[idx][model_prediction[idx].argmax()].exp()/model_prediction[idx].exp().sum())))
        '''
        prediction_value=prediction[idx]
        p_x=model_prediction[idx][model_prediction[idx].argmax()].exp()
        p_sum=model_prediction[idx].exp().sum()
        prediction_score=100*float(model_prediction[idx][model_prediction[idx].argmax()].exp()/model_prediction[idx].exp().sum())
        #print('prediction_value:',prediction_value)
        #print('prediction_score:',prediction_score)
        #print('分子:',p_x)
        #print('分母：',p_sum)
        #uu=pr
        #print("torch.argmax:",torch.argmax(target[idx]))
        if len(genes_features[prediction_value])==1:#有值，但还没有添加预测分数和特征值，只有标签#prediction_value代表第几种疾病
            if prediction_value!=torch.argmax(target[idx]).cpu().detach()  :
                print('不合格****************:',prediction_value)
            if prediction_value==torch.argmax(target[idx]).cpu().detach()  :#预测正确
                genes_features[prediction_value].append(prediction_score)
                print('合格：',prediction_value)
                relevance_score_for_every_pixel=evidence_for_class[prediction[idx]][idx]
                #print('relevance_score_for_every_pixel.shape:',relevance_score_for_every_pixel.tolist())
                relevance_score_for_every_pixel=torch.from_numpy(relevance_score_for_every_pixel)
                index=torch.topk(relevance_score_for_every_pixel,30,largest=True)#基因个数50#####150
                #print('pixel_sorted:',index)
                genes_features[prediction_value].append(index.indices)#添加前50基因特征
        
        else: 
            if genes_features[prediction_value][1]<prediction_score:#如果值比已有值大，说明预测更准确
                if prediction_value==torch.argmax(target[idx]).cpu().detach():
                    print('&&&&&&&&&&&&&&&&&&&合   格&&&&&&&&&&&&&&:',prediction_value)
                    genes_features[prediction_value].pop(2)#先删除特征值
                    genes_features[prediction_value].pop(1)#先删除预测分数
                
                    genes_features[prediction_value].append(prediction_score)
        
                    relevance_score_for_every_pixel=evidence_for_class[prediction[idx]][idx]
                    #print('relevance_score_for_every_pixel.shape:',relevance_score_for_every_pixel.tolist())
                    relevance_score_for_every_pixel=torch.from_numpy(relevance_score_for_every_pixel)
                    index=torch.topk(relevance_score_for_every_pixel,30,largest=True)#基因个数50  200
                    #print('pixel_sorted:',index)
                    genes_features[prediction_value].append(index.indices)
print('qiu_____genes_features.shape:',genes_features)



MLP(
  (bn0): BatchNorm1d(3301, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=3301, out_features=800, bias=True)
  (bn1): BatchNorm1d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=800, out_features=100, bias=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=100, out_features=9, bias=True)
  (bn3): BatchNorm1d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
relevance
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(3, device='cuda:0')
torch.argmax(predict): tensor(4, device='cuda:0')
torch.argmax(predict): tensor(0, device='cuda:0')
torch.argmax(predict): tensor(2, device='cuda:0')
torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(3, device='cuda:0')

torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(2, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(4, device='cuda:0')
torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(2, device='cuda:0')
torch.argmax(predict): tensor(1, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(1, device='cuda:0')
torch.argmax(predict): tensor(4, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(3, device='cuda:0')
torch.argmax(predict): tensor(2, device='cuda:0')
torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(2, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')


torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(5, device='cuda:0')
torch.argmax(predict): tensor(5, device='cuda:0')
torch.argmax(predict): tensor(0, device='cuda:0')
torch.argmax(predict): tensor(4, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(1, device='cuda:0')
torch.argmax(predict): tensor(5, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(1, device='cuda:0')
torch.argmax(predict): tensor(2, device='cuda:0')
torch.argmax(predict): tensor(3, device='cuda:0')
torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(5, device='cuda:0')
torch.argmax(predict): tensor(5, device='cuda:0')


In [3]:
for i in range(9):
    
    qiu_genes_tensor=genes_features[i][2]
    if i==0:
        mm=qiu_genes_tensor
    else:
        mm=torch.vstack((mm,qiu_genes_tensor))
print(mm.shape)
qiu_genes_numpy=mm.detach().numpy()
qiu_genes_df=pd.DataFrame(qiu_genes_numpy)
qiu_genes_df.to_csv('dataset/qiuguan/origin_800/LRP/30/qiu_genes_df.csv')

torch.Size([9, 30])


In [12]:
###########如果不能为每个疾病找到特征基因，就反复点击这段代码，直到找到为止
testset_xiaoguan  = KZDatasetTest(csv_path='dataset/qiuguan/origin_800/xiaoguan/test_info.csv')
   
test_loader_xiaoguan = DataLoader(
         dataset=testset_xiaoguan,
         #transform=torchvision.transforms.ToTensor(),
         
         batch_size=nfm_config['batch_size'],
         shuffle=True
        
     )
#小管特征        
for data, target in test_loader_xiaoguan:############小管

    data, target = data.to(device), target.to(device)
    #targets=torch.max(targets,1)[1]###################
    #print('data:',data.shape)
    batch_size = int(data.size()[0])
    #print('batch_size:',batch_size)#=20
    evidence_for_class = []
    #print("target:",target.shape)
    #print('target:',target[3])
    # Overlay with noise 
    # data[0] += 0.25 * data[0].max() * torch.Tensor(np.random.randn(28*28).reshape(1, 28, 28))
    #model_prediction, true_relevance = inn_model.innvestigate(in_tensor=data)

    for i in range(9):#10类
    # Unfortunately, we had some issue with freeing pytorch memory, therefore 
    # we need to reevaluate the model separately for every class.
        model_prediction, input_relevance_values = inn_model.innvestigate2(in_tensor=data, rel_for_class=i,target=target)
        evidence_for_class.append(input_relevance_values)
    #print('input_relevance_values:',input_relevance_values.shape)
    #print('evidence_for_class:',len(evidence_for_class))
    evidence_for_class = np.array([elt.numpy() for elt in evidence_for_class])
    #print('evidence_for_class:',evidence_for_class.shape)#[10,20,784]
    for idx, example in enumerate(data):#batch 中的每一个样本
        #print('example:',example.shape)
        prediction = np.argmax(model_prediction.cpu().detach(), axis=1)#
        #print('prediction[idx]:',prediction[idx])
        #print(evidence_for_class[prediction[idx]][idx])
        #fig, axes = plt.subplots(3, 5)
        '''
        fig.suptitle("Prediction of model: " + str(prediction[idx]) + "({0:.2f})".format(
            100*float(model_prediction[idx][model_prediction[idx].argmax()].exp()/model_prediction[idx].exp().sum())))
        
        '''
        
        prediction_value=prediction[idx]
        p_x=model_prediction[idx][model_prediction[idx].argmax()].exp()
        p_sum=model_prediction[idx].exp().sum()
        prediction_score=100*float(model_prediction[idx][model_prediction[idx].argmax()].exp()/model_prediction[idx].exp().sum())
        #print('prediction_value:',prediction_value)
        #print('prediction_score:',prediction_score)
        #print('分子:',p_x)
        #print('分母：',p_sum)
        #uu=pr
        #print("torch.argmax:",torch.argmax(target[idx]))
        if len(genes_features[prediction_value])==3:#有小球的值，但还没有添加小管预测分数和特征值，只有标签
            if prediction_value!=torch.argmax(target[idx]).cpu().detach()  :
                print('不合格****************:',prediction_value)
            if prediction_value==torch.argmax(target[idx]).cpu().detach()  :#预测正确
                genes_features[prediction_value].append(prediction_score)
                print('合格：',prediction_value)
                relevance_score_for_every_pixel=evidence_for_class[prediction[idx]][idx]
                #print('relevance_score_for_every_pixel.shape:',relevance_score_for_every_pixel.tolist())
                relevance_score_for_every_pixel=torch.from_numpy(relevance_score_for_every_pixel)
                index=torch.topk(relevance_score_for_every_pixel,30,largest=True)#基因个数50
                #print('pixel_sorted:',index)
                genes_features[prediction_value].append(index.indices)#添加前50基因特征
        
        else: 
            if genes_features[prediction_value][3]<prediction_score:#如果值比已有值大，说明预测更准确
                if prediction_value==torch.argmax(target[idx]).cpu().detach():
                    print('&&&&&&&&&&&&&&&&&&&合   格&&&&&&&&&&&&&&:',prediction_value)
                    genes_features[prediction_value].pop(4)#先删除特征值
                    genes_features[prediction_value].pop(3)#先删除预测分数
                
                    genes_features[prediction_value].append(prediction_score)
        
                    relevance_score_for_every_pixel=evidence_for_class[prediction[idx]][idx]
                    #print('relevance_score_for_every_pixel.shape:',relevance_score_for_every_pixel.tolist())
                    relevance_score_for_every_pixel=torch.from_numpy(relevance_score_for_every_pixel)
                    index=torch.topk(relevance_score_for_every_pixel,30,largest=True)#基因个数50
                    #print('pixel_sorted:',index)
                    genes_features[prediction_value].append(index.indices)
    print('guan______________genes_features.shape:',genes_features)       
        
    '''
        #写个算法，看多少个像素就能把一个图像全都表达出来
        relevance_score_for_every_pixel=evidence_for_class[prediction[idx]][idx]
        #relevance_score_for_every_pixel.sort()
        row,=relevance_score_for_every_pixel.shape
        num=0
        for i in range(row):
            if relevance_score_for_every_pixel[i,]<0.005:
                evidence_for_class[prediction[idx]][idx][i,]=1
            else:
                num=num+1
        print("num:",num)       
        #print('evi:',evidence_for_class[prediction[idx]][idx])
        vmin = np.percentile(evidence_for_class[:, idx], 50)
        vmax = np.percentile(evidence_for_class[:, idx], 99.9)
        
        axes[0, 2].imshow(example[0])
        
        axes[0, 2].set_title("Input (" + str(int(target[idx]))+ ")")
        #axes[0, 3].imshow(evidence_for_class[prediction[idx]][idx][0], vmin=vmin,
                          #vmax=vmax, cmap="hot")#【0】
        axes[0, 3].imshow(evidence_for_class[prediction[idx]][idx].reshape(28,28), vmin=vmin,
                          vmax=vmax, cmap="hot")
        axes[0, 3].set_title("Pred. Evd.")
        for ax in axes[0]:
            ax.set_axis_off()
        for j, ax in enumerate(axes[1:].flatten()):
            #im = ax.imshow(evidence_for_class[j][idx][0], cmap="hot", vmin=vmin,
                         # vmax=vmax)#【0】。reshape
            im = ax.imshow(evidence_for_class[j][idx].reshape(28,28), cmap="hot", vmin=vmin,
                          vmax=vmax)
            ax.set_axis_off()
            ax.set_title("Evd. " + str(j))
        fig.colorbar(im, ax=axes.ravel().tolist())
        plt.show()
    '''
        
        
    break
        # fig.savefig("./mnist_example{0}.png".format(idx))

torch.argmax(predict): tensor(3, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(0, device='cuda:0')
torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(1, device='cuda:0')
torch.argmax(predict): tensor(1, device='cuda:0')
torch.argmax(predict): tensor(4, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(5, device='cuda:0')
torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(0, device='cuda:0')
torch.argmax(predict): tensor(4, device='cuda:0')
torch.argmax(predict): tensor(3, device='cuda:0')
torch.argmax(predict): tensor(3, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(0, device='cuda:0')
torch.argmax(predict): tensor(6, device='cuda:0')


In [13]:
for i in range(9):
    
    guan_genes_tensor=genes_features[i][4]
    if i==0:
        mm=guan_genes_tensor
    else:
        mm=torch.vstack((mm,guan_genes_tensor))
print(mm.shape)
guan_genes_numpy=mm.detach().numpy()
guan_genes_df=pd.DataFrame(guan_genes_numpy)
guan_genes_df.to_csv('dataset/qiuguan/origin_800/LRP/30/guan_genes_df.csv')

torch.Size([9, 30])


In [14]:
#保存筛选的特征基因

import itertools

qiu_guan_all=[]

for i in range(9):
    #print('i:',i)
    qiu_features=genes_features[i][2]
    guan_features=genes_features[i][4]
    #print(qiu_features)
    qiu_features=qiu_features.detach().numpy()
    guan_features=guan_features.detach().numpy()
    qiu_guan_features = list(set(itertools.chain(*[qiu_features,guan_features])))
    #print(qiu_guan_features)
    #qiu_guan_features.append(i)
    qiu_guan_all.append(qiu_guan_features)
#qiu_guan_all.append('label')
#qiu_guan_all.append(10478)


qiu_guan_all_np=np.array(qiu_guan_all)
qiu_guan_all_df=pd.DataFrame(qiu_guan_all_np)
qiu_guan_all_df.to_csv('dataset/qiuguan/origin_800/LRP/30/qiu_guan_all_selected_features.csv')

/home/zhengfang/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [15]:
#求出所有基因的个数，基因名数
##########对所有疾病的特征基因求交集
import itertools
import numpy as np
import pandas as pd 
f_genes='dataset/qiuguan/origin_800/LRP/30/qiu_guan_all_selected_features.csv'
f_genes_df=pd.read_csv(f_genes,sep=',',header=None)
#print(f_genes_df)
f_genes_df=f_genes_df.iloc[1:,1:]
#print(f_genes_df)
#f_genes_df=f_genes_df.astype(int)
#f_genes_df=f_genes_df.values

qiu_guan_all=f_genes_df
qiu_guan_all=np.array(qiu_guan_all)
##qiu_guan_all.dtype='int'
#print(qiu_guan_all)
for i in range(9):
    #print('i:',i)
    qiu_guan_all_list=qiu_guan_all[i][0]
    #qiu_guan=qiu_guan[:-1]
    #print(qiu_guan)
    #=qiu_guan_all_list[0]
    qiu_guan_all_list_to_int = qiu_guan_all_list[1:-1]#去掉方括号，已经不是列表了，把方括号整体当作一个字符了
    
    qiu_guan_all_list_to_int=qiu_guan_all_list_to_int.split(',')#按，分割
    qiu_guan_all_list_to_int=[int(j) for j in qiu_guan_all_list_to_int]
    #print(qiu_guan_all_list_to_int)
    #guan_features=genes_features[i][4]
    #print(qiu_features)
    #qiu_features=qiu_features.detach().numpy()
    #guan_features=guan_features.detach().numpy()
    #qiu_guan=[str(i) for i in qiu_guan]
    if i==0:
        qiu_guan_final=qiu_guan_all_list_to_int
    else:
        
        #qiu_guan_final=qiu_guan_final.
        qiu_guan_final = list(set(itertools.chain(*[qiu_guan_final,qiu_guan_all_list_to_int])))###########将球管合并，删除重复基因
#print(len(qiu_guan_final))
qiu_guan_final_len=len(qiu_guan_final)        

        #print(qiu_guan_features)
        #qiu_guan_features.append(i)
        #qiu_guan_all.append(qiu_guan_final)
#qiu_guan_all.append('label')
#qiu_guan_all.append(10478)
#print(qiu_guan_all)

qiu_guan_final=np.array(qiu_guan_final).reshape(1,qiu_guan_final_len)
print(qiu_guan_final.shape)
qiu_guan_final=pd.DataFrame(qiu_guan_final)
qiu_guan_final.to_csv('dataset/qiuguan/origin_800/LRP/30/qiu_guan_all_names_new_df_1.csv')

(1, 160)


In [16]:
import os
import time
import argparse
import numpy as np
import pandas as pd 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
#3from tensorboardX import SummaryWriter
import torch.nn.functional as F  # 激励函数的库
#import network
import config
import evaluate
import data_utils
import Trainer

def one_hot_smoothing(labels, classes, label_smoothing=0.2):
    #n = len(labels)
    n=labels.shape[0]
    eoff = label_smoothing / classes
    output = np.ones((n, classes), dtype=np.float32) * eoff
    for row, label in enumerate(labels):
        output[row, label] = 1 - label_smoothing + eoff
        #print("row:",row,"label:",label)
    return output

def one_hot(labels, classes):
    n = len(labels)
    #eoff = label_smoothing / classes
    output = np.zeros((n, classes), dtype=np.float32)
    for row, label in enumerate(labels):
        output[row, label] = 1
        #print("row:",row,"label:",label)
    return output


import pandas as pd 

from sklearn.model_selection import train_test_split


'''
all=pd.read_csv('dataset/qiuguan/origin_800/new_df_df_800_non_encode.csv',sep=',',header=None)
#all=pd.read_csv('dataset/qiuguan/orign/union_all_qiuguan.csv',sep=',')
#print(all)
#all.rename(columns={'labels.1': 'labels'}, inplace=True)
all=all.iloc[:,1:]
print('all:',all)
print('shape:',all.shape)
#print(all)
#prin

#X=X.values
all_rows,all_cols=all.shape

'''

#######训练集 ：合起来：一部分
all_qiuguan=pd.read_csv('dataset/qiuguan/origin_800/xiaoqiu_xiaoguan/train_val_info.csv',sep=',',header=None)
#all=pd.read_csv('dataset/qiuguan/orign/union_all_qiuguan.csv',sep=',')
#print(all)
#all.rename(columns={'labels.1': 'labels'}, inplace=True)
all_qiuguan=all_qiuguan.iloc[:,1:]
print('all:',all_qiuguan)
print('shape:',all_qiuguan.shape)
#print(all)
#prin

#X=X.values
all_rows,all_cols=all_qiuguan.shape

###########测试集：分两部分：小球和小管

all_xiaoguan=pd.read_csv('dataset/qiuguan/origin_800/xiaoguan/test_info.csv',sep=',',header=None)
#all=pd.read_csv('dataset/qiuguan/orign/union_all_qiuguan.csv',sep=',')
#print(all)
#all.rename(columns={'labels.1': 'labels'}, inplace=True)
all_xiaoguan=all_xiaoguan.iloc[:,1:]
print('all_xiaoguan:',all_xiaoguan)
print('shape:',all_xiaoguan.shape)
#print(all)
#prin

#X=X.values
#all_rows,all_cols=all.shape


#all_rows,all_cols=all.shape

###########测试集：分两部分：小球和小管

all_xiaoqiu=pd.read_csv('dataset/qiuguan/origin_800/xiaoqiu/test_info.csv',sep=',',header=None)
#all=pd.read_csv('dataset/qiuguan/orign/union_all_qiuguan.csv',sep=',')
#print(all)
#all.rename(columns={'labels.1': 'labels'}, inplace=True)
all_xiaoqiu=all_xiaoqiu.iloc[:,1:]
print('all_xiaoqiu:',all_xiaoqiu)
print('shape:',all_xiaoqiu.shape)



#all_rows_name=all.index=['a'+str(i) for i in range(all_rows)]
#print(rows_name)

#print(all)
column_name=pd.read_csv('dataset/qiuguan/origin_800/LRP/30/qiu_guan_all_names_new_df_1.csv',sep=',',header=None)
print('column_names:',column_name)
column_name=column_name.iloc[1:,1:]
row_num,col_num=column_name.shape
print('column_names:',column_name)

print('row_num,col_num:',row_num,col_num)
#=column_name.astype(int)
column_name=np.array(column_name).reshape(col_num).tolist()
print('xxxxxxxxxxxxxxxx:',column_name)
#column_name_index=[i-1 for i in column_name]############################################
column_name_index=[i for i in column_name]
#column_name[-1]='10478'
#print('xxxxxxxxxxxxxxxx:',column_name)
#column_name=[int(i) for i in range(column_name)]
#column_name=list(map(int,column_name))
#print('column_names:',column_name)
#rows_name,col_name=column_name.shape
#column_name=column_name.values.reshape(rows_name).tolist()
column_name_index.append(3301)
#all=all.loc[all_rows_name,column_name]
all_qiuguan=all_qiuguan.iloc[:,column_name_index]
print(all_qiuguan.shape)
print(all_qiuguan)

all_xiaoguan=all_xiaoguan.iloc[:,column_name_index]
all_xiaoqiu=all_xiaoqiu.iloc[:,column_name_index]
print(all_xiaoqiu)
print(all_xiaoguan)
'''
X=all.iloc[1:,:-1]
#print(column_name)

#X=all.iloc[]

#X.to_csv('dataset/qiuguan/origin_800/final_100/X.csv')
#print(X)

y=all.iloc[1:,-1]
y=pd.DataFrame(y,dtype=int)

'''

#print(y)
#print(y.dtype)
#y=y.values

#print(y)

"""
y=pd.read_csv('dataset/gene_247/data/guan/guan_label.csv',sep=',')
y=y.values

"""

'''
#y=np.array(y)
train_val_data,test_data,train_val_label,test_label=train_test_split(X,y,test_size=0.2,stratify=y,random_state=3)

#print(train_val_data)
train_val_info=pd.concat([train_val_data,train_val_label],axis=1)
test_info=pd.concat([test_data,test_label],axis=1)

print(train_val_info)

'''




all_qiuguan.to_csv('dataset/qiuguan/origin_800/LRP/30/selected_train_val_info.csv')
all_xiaoguan.to_csv('dataset/qiuguan/origin_800/LRP/30/selected_xiaoguan_test_info.csv')
all_xiaoqiu.to_csv('dataset/qiuguan/origin_800/LRP/30/selected_xiaoqiu_test_info.csv')

/home/zhengfang/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249

all:             1                  2                  3                  4     \
0           RHOA               STX2              CISD1              WDR11   
1    11.02328453        6.234847141        10.16119932        7.733762949   
2    11.12296085        6.050914631        10.04924825        7.506624491   
3    11.34208097  6.201381637999999        9.584881578  7.335857927999999   
4     11.4064088        6.377802689  9.637117742000001        7.071728444   
..           ...                ...                ...                ...   
541       12.124            8.98472            8.99447             8.5349   
542      12.7133            8.60548             8.8762            9.57229   
543      12.2895            8.69831            9.33649             9.1396   
544      12.2937            8.91005            8.67297            9.03033   
545      12.4217            8.97756            8.72483            8.78966   

            5            6            7            8            9     

all_xiaoqiu:            1                  2            3                  4     \
0          RHOA               STX2        CISD1              WDR11   
1   12.33147733        8.843523187  9.435926205        9.009858572   
2    12.1801838  8.631155163999999  9.379102965  8.340756477000001   
3   11.98135389  8.782033697000001  9.156410126         8.50322152   
4   12.27318789        8.327898846  9.887579274  8.634472367999999   
..          ...                ...          ...                ...   
66  12.15324978        8.550678572  9.548931032        8.448183924   
67  12.50080673        8.632233391   9.36522456        9.203636018   
68  12.21194258        8.845552827  8.872258606        8.688882053   
69  12.45519228        8.813248733  8.660720575        8.951006739   
70  12.44740614        8.842976231  9.422864349            8.98144   

                 5                  6            7                  8     \
0               SCYL2            MGC2889       CCDC47               KL